# Backup & Restore

Use `AdventureWorks2017` as a database inside this notebook.


1. Review the database logging settings for both `AdventureWorks2017` and `AdventureWorksDW2017`
   * `AdventureWorks2017` should be set to `Full logging`
   * `AdventureWorksDW2017` should be set to `Simple logging`

You can either review this via the GUI, or in T-SQL

* SSMS: right-click the database, view "Properties"
* Azure Data Studio: right-click the database, click "Manage"
* T-SQL: 
```sql
SELECT name, recovery_model_desc 
FROM sys.databases;
```

In [2]:
SELECT name, recovery_model_desc  
   FROM sys.databases;  

(7 rows affected)

Total execution time: 00:00:00.071

name,recovery_model_desc
master,SIMPLE
tempdb,SIMPLE
model,FULL
msdb,SIMPLE
AdventureWorks2017,FULL
AdventureWorksDW2017,SIMPLE
AdventureWorksLT2017,SIMPLE


2. Select (and note) the current time (try to use a SQL command returning the current date and time)


In [7]:
SELECT GETDATE();

(1 row affected)

Total execution time: 00:00:00.029

(No column name)
2019-09-24 10:07:29.667


3. Insert a new salesperson in `AdventureWorks2017` by adding data to the following two tables:
   * In `Person.BusinessEntity`: add a row
     * rowguid: Use the `NEWID()` function to generate a new GUID
     * ModifiedDate: Use `GETDATE()` to get the current date
   * In `AdventureWorks.Sales.SalesPerson`:
     * BusinessEntityID: look up (manually) the ID just generated in `Person.BusinessEntity.BusinessEntityID`
     * TerritoryID: 7 (from `Sales.SalesTerritory`: France)
     * SalesQuota, Bonus, CommissionPct, SalesYTD, SalesLastYear: use the values from SalesPerson with BusinessEntityID 290, add 10%
       * Bonus (and probably the easiest way): gather the correct values within a SELECT-statement, then use INSERT-SELECT to add the row
     * rowguid: Use the `NEWID()` function to generate a new GUID
     * ModifiedDate: Use `GETDATE()` to get the current date

4. Insert a new employee in `AdventureWorksDW2017`. Use the same values as employee with dimension ID 269

5. Select (and note) the current time (yes, again)

(10 rows affected)

Total execution time: 00:00:00.1029954

orderyear,SalesPersonID,NumOfOrders
2012,277,166
2012,275,148
2013,279,159
2013,275,175
2013,276,162
2012,276,151
2013,277,185
2012,279,153
2013,289,170
2012,289,111


6. Delete (semi-accidentally) all rows from
   * `dbo.FactResellerSales` in `AdventureWorksDW2017`
   * `Sales.SalesOrderDetail` in `AdventureWorks2017`


## Restore backups

We are now in a situation where we "accidentally" removed a large number of rows in both databases.
In both cases, a "full backup" is available. This means we can roll back both databases, but with one major difference:

* with **full recovery model** (aka *full logging*), we can roll back to **any point in time**. This is called *Point In Time Recovery*.
* with **simple logging**, we can only roll back to the **exact situation on the moment of backup**.

### Restore the AdventureWorks database

The `AdventureWorks2017` database has a **full recovery model**. This means:

* larger transaction log files (.ldf)
* more IOPS used (writes and reads per second to disk)
* point in time recovery possible

The query below will show all transaction log files, displaying the current size (on disk the file will be larger, as there should always be room to store more transactions)


In [8]:
SELECT 'AdventureWorks2017', file_id, name, type_desc, physical_name, size, max_size
FROM AdventureWorks2017.sys.database_files

UNION ALL

SELECT 'AdventureWorksDW2017', file_id, name, type_desc, physical_name, size, max_size
FROM AdventureWorksDW2017.sys.database_files


(4 rows affected)

Total execution time: 00:00:02.495

(No column name),file_id,name,type_desc,physical_name,size,max_size
AdventureWorks2017,1,AdventureWorks2017,ROWS,C:\Program Files\Microsoft SQL Server\MSSQL14.MSSQLSERVER\MSSQL\DATA\AdventureWorks2017.mdf,33792,-1
AdventureWorks2017,2,AdventureWorks2017_log,LOG,C:\Program Files\Microsoft SQL Server\MSSQL14.MSSQLSERVER\MSSQL\DATA\AdventureWorks2017_log.ldf,9216,268435456
AdventureWorksDW2017,1,AdventureWorksDW2017,ROWS,C:\Program Files\Microsoft SQL Server\MSSQL14.MSSQLSERVER\MSSQL\DATA\AdventureWorksDW2017.mdf,17408,-1
AdventureWorksDW2017,2,AdventureWorksDW2017_log,LOG,C:\Program Files\Microsoft SQL Server\MSSQL14.MSSQLSERVER\MSSQL\DATA\AdventureWorksDW2017_log.ldf,9216,268435456


To do a point-in-time (PIT) restore, start SSMS.
From there, follow the instructions on [https://docs.microsoft.com/en-us/sql/relational-databases/backup-restore/restore-a-sql-server-database-to-a-point-in-time-full-recovery-model?view=sql-server-2017](https://docs.microsoft.com/en-us/sql/relational-databases/backup-restore/restore-a-sql-server-database-to-a-point-in-time-full-recovery-model?view=sql-server-2017) under the heading **Using SQL Server Management Studio**.

* Restore to the point *before* the "accidental" deletion of `SalesOrderDetail`, but *after* the inserting of the new `SalesPerson`. 
* Make sure the `WITH OVERWRITE` option is selected
* Make sure active connections are closed


### Restore the AdventureWorksDW database

Given the explanation about *full recovery model*, my expectation is that this I cannot do a point-in-time recovery for the `AdventureWorksDW` database, because it doesn't log all transactions in between. 

To test this assumption, try to do the same restore (before deleting the fact, but after adding the new employee) on AdventureWorksDW.

If this is not possible, do a restore of the full backup, and check that the new employee has disappeared from the DimEmployee table. You can do this either via the GUI in SSMS, or try to find the T-SQL commands to do it using T-SQL.